<a href="https://colab.research.google.com/github/CIT-NLP/NLP-final-project/blob/main/LogRegwithSMOTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown https://drive.google.com/uc?id=1gRM0y476quq727mTJJpE2B-42g7OhtaK
!gdown https://drive.google.com/uc?id=14bZuqXhhr6P7MabEuVb__C7gwbIUagF9

Downloading...
From: https://drive.google.com/uc?id=1gRM0y476quq727mTJJpE2B-42g7OhtaK
To: /content/data.csv
100% 209M/209M [00:00<00:00, 270MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wHRVN7l6quMFTG7seVS0pHh7ULrc78ug
To: /content/processed_dataframe.csv
100% 180M/180M [00:01<00:00, 175MB/s]


In [ ]:
import pandas as pd
from ast import literal_eval

df_1 = pd.read_csv("./data.csv")
df_2 = pd.read_csv("./processed_dataframe.csv")

df  = df_2

# interpret cols as lists instead of strings
for col in ['unigrams', 'sentences', 'lem_sent', 'lem_sent_join']: 
  df[col] = df[col].apply(literal_eval)

df

KeyError: ignored

In [ ]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# cnt_pro = df['genre'].value_counts()
cnt_pro = df['genre_new'].value_counts()
plt.figure(figsize=(12,4))
sns.barplot(cnt_pro.index, cnt_pro.values, alpha=0.8)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('genres', fontsize=12)
plt.xticks(rotation=90)
plt.show();

In [ ]:
from sklearn.model_selection import train_test_split
import gensim
from gensim.models.doc2vec import TaggedDocument

train, test = train_test_split(df, test_size=0.2, random_state=42)
# train_tagged = train.apply(lambda r: TaggedDocument(words=r['unigrams'], tags=r.genre), axis=1)
# test_tagged = test.apply(lambda r: TaggedDocument(words=r['unigrams'], tags=r.genre), axis=1)
train_tagged = train.apply(lambda r: TaggedDocument(words=r['unigrams'], tags=r.genre_new), axis=1)
test_tagged = test.apply(lambda r: TaggedDocument(words=r['unigrams'], tags=r.genre_new), axis=1)

train_tagged.values[0].tags

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()
cores

In [ ]:
from gensim.models import Doc2Vec

model_dbow = Doc2Vec(dm=0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

In [ ]:
from sklearn import utils

for epoch in range(5):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

In [ ]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags, model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [ ]:
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

In [ ]:
import numpy as np

X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [ ]:
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state = 0)
os_smote_X,os_smote_Y = os.fit_resample(X_train, y_train)


print(X_train.shape)

print(os_smote_X.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression


logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

In [ ]:
y_pred[:1000]